# Node Classification
AutoGL supports multiple graph related tasks, including node classification. 

In this file we will give you a simple example to show how to use AutoGL to do the node classification task.

## Import libraries
First, you should import some libraries and you can set the random seed before you split the dataset and train the model.

In [1]:
import yaml
import random
import torch.backends.cudnn
import numpy as np

from autogl.datasets import build_dataset_from_name
from autogl.solver import AutoNodeClassifier
from autogl.module import Acc
from autogl.backend import DependentBackend

# set random seed
random.seed(2022)
np.random.seed(2022)
torch.manual_seed(2022)
if torch.cuda.is_available():
    torch.cuda.manual_seed(2022)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Load Dataset
AutoGL provides a very convenient interface to obtain and partition common datasets, such as cora, citeseer, and amazon_computers, etc.

You just need to give the name of the dataset you want and AutoGL will return the dataset.

In this example, we evaluate model on Cora dataset in the semi-supervised node classification task.

In [2]:
dataset = build_dataset_from_name('cora')

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


## Initialize Solver
After obtaining the dataset, we need to initialize the model.

However, as AutoGL provides a convenient method to use HPO to better optimize the model, we can train the model through the solver class provided by AutoGL.

Solver in AutoGL usually uses a config file for lazy initialization. The format of the config file can be found in the `../config` folder for examples, or you can read our tutorial for some help.

In [3]:
label = dataset[0].nodes.data['y' if DependentBackend.is_pyg() else 'label']
num_classes = len(np.unique(label.numpy()))

configs = yaml.load(open('../configs/nodeclf_gcn_benchmark_small.yml', "r").read(), Loader=yaml.FullLoader)
autoClassifier = AutoNodeClassifier.from_config(configs)

## Train
After the initialization is finished, you can use the interface provided by AutoGL to optimize the model through HPO.

In [4]:
# time limit is the seconds limited for training the model
# evaluation method is the metric to evaluate the performance
autoClassifier.fit(dataset, time_limit=3600, evaluation_method=[Acc])

[2022-10-24 18:55:27] INFO (NodeClassifier/MainThread) Use the default train/val/test ratio in given dataset
HPO Search Phase:



100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:09<00:00,  1.40s/it]


[2022-10-24 18:56:37] INFO (HPO/MainThread) Best Parameter:
[2022-10-24 18:56:37] INFO (HPO/MainThread) Parameter: {'trainer': {'max_epoch': 165, 'early_stopping_round': 18, 'lr': 0.014545893271287733, 'weight_decay': 0.0001682578213292401}, 'encoder': {'num_layers': 2, 'hidden': [42], 'dropout': 0.6019468841551312, 'act': 'tanh'}, 'decoder': {}} acc: 0.806 higher_better


## Evaluation
After training, you can evaluate the model.

In [5]:
autoClassifier.get_leaderboard().show()
# you can also provided the metric here!
acc = autoClassifier.evaluate(metric="acc")
print("test acc: {:.4f}".format(acc))

+-------------------------------------------------------------------------+-------+
| name                                                                    |   acc |
+=========================================================================+=======+
| decoder: None                                                           | 0.806 |
| early_stopping_round: 18                                                |       |
| encoder: <autogl.module.model.dgl.gcn.AutoGCN object at 0x7fa3186d0b50> |       |
| learning_rate: 0.014545893271287733                                     |       |
| max_epoch: 165                                                          |       |
| optimizer: !!python/name:torch.optim.adam.Adam ''                       |       |
| trainer_name: NodeClassificationFullTrainer                             |       |
| _idx0                                                                   |       |
+-------------------------------------------------------------------------+-